In [1]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split

caltech_dir = "C:/Users/User/Desktop/semiconductor2/train"
categories = ["0", "1","2", "3", "4", "5", "6","7", "8","9","10","11","12","13","14"]
nb_classes = len(categories)

image_w = 64
image_h = 64

pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

        if i % 700 == 0:
            print(cat, " : ", f)

X = np.array(X)
y = np.array(y)
#1 0 0 0 이면 airplanes
#0 1 0 0 이면 buddha 이런식


X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save("C:/Users/interbird/Desktop/semiconductor2/numpyData", xy)


print("ok", len(y))

0  파일 길이 :  87
0  :  C:/Users/interbird/Desktop/semiconductor2/train/0\20200811_094805_001.jpg
1  파일 길이 :  62
1  :  C:/Users/interbird/Desktop/semiconductor2/train/1\20200811_092330_001.jpg
2  파일 길이 :  142
2  :  C:/Users/interbird/Desktop/semiconductor2/train/2\20200811_095448_001.jpg
3  파일 길이 :  126
3  :  C:/Users/interbird/Desktop/semiconductor2/train/3\20200811_145817_001.jpg
4  파일 길이 :  134
4  :  C:/Users/interbird/Desktop/semiconductor2/train/4\20200811_151430_001.jpg
5  파일 길이 :  105
5  :  C:/Users/interbird/Desktop/semiconductor2/train/5\20200812_091205_001.jpg
6  파일 길이 :  110
6  :  C:/Users/interbird/Desktop/semiconductor2/train/6\20200812_114653_001.jpg
7  파일 길이 :  85
7  :  C:/Users/interbird/Desktop/semiconductor2/train/7\20200812_094114_001.jpg
8  파일 길이 :  126
8  :  C:/Users/interbird/Desktop/semiconductor2/train/8\20200812_142222_001.jpg
9  파일 길이 :  113
9  :  C:/Users/interbird/Desktop/semiconductor2/train/9\20200812_165139_001.jpg
10  파일 길이 :  158
10  :  C:/Users/interbird/

C:\Users\interbird\anaconda3\envs\test\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [14]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K

import tensorflow as tf

# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth = True
# session = tf.compat.v1.Session(config=config)

data2 = np.load('C:/Users/User/Desktop/semiconductor2/numpyData.npy', allow_pickle=True)
X_train, X_test, y_train, y_test = data2


#X_train, X_test, y_train, y_test = np.load('./numpy_data/multi_image_data.npy')
print(X_train.shape)
print(X_train.shape[0])

(1394, 64, 64, 3)
1394


In [3]:
categories = ["0", "1","2", "3", "4", "5", "6","7", "8","9","10","11","12","13","14"]
nb_classes = len(categories)

#일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255

In [10]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3), padding='same', activation="relu"))
model.add(Conv2D(64,(3,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(nb_classes, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


model_dir = './model'
hdf5_file = './hd5'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

if os.path.exists(hdf5_file):
    model.load_weights(hdf5_file)
else:
    model.fit(X_train, y_train, batch_size=32, epochs = 10)
    model.save_weights(hdf5_file)

model_path = model_dir + '/multi_img_classification.model'
checkpoint = ModelCheckpoint(filepath = model_path, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [6]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 15, 15, 64)       

In [7]:
score = model.evaluate(X_test, y_test)
print("loss = ", score[0])
print("accuracy= ", score[1])

465/465 [==============================] - 1s 1ms/step
loss =  0.0005332381527129841
accuracy=  1.0


In [14]:
test_image = 'C:/Users/User/Desktop/semiconductor2/train/11/0.jpg'

img = Image.open(test_image)
img = img.convert("RGB")
img = img.resize((64,64))
data = np.asarray(img)

X = np.array(data)
X = X.astype("float")/256
X = X.reshape(-1,64,64,3)

pred = model.predict(X)
result = [np.argmax(value) for value in pred]
print('New Data Category : ', categories[result[0]])
#print(pred)

New Data Category :  11
